# Casa Grande KH-9 Mapping Camera preprocessing 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import usgsxplore
import os
import history
import hipp

## Settings

In [4]:
# all paths are set here 
PROJECT_DIRECTORY = "/home/godinlu/Documents/history-dataset-v2/casa_grande/kh9mc/1978_03_25"
RAW_IMAGES = os.path.join(PROJECT_DIRECTORY, "raw_images")
PREPROCESSED_IMAGES = os.path.join(PROJECT_DIRECTORY, "preprocessed_images")

QUICKVIEW_FACTOR = 0.10

# for the downloading via USGS
username = os.getenv("USGS_USERNAME") or input("Enter your USGS username: ")
token = os.getenv("USGS_TOKEN") or input("Enter your USGS token: ")

## The Dataset

The dataset consists of **4 satelites images** captured using a **Mapping Camera**. The images were acquired on **March 25, 1978**, and cover the **Casa Grande Glacier** region in the United States.

They are available for download via the [Earth Explorer](https://earthexplorer.usgs.gov/) plateform, which's provid images spited in two part due to their size.


In [6]:
df = history.dataquery.get_casa_grande_kh9mc()
df

,Entity ID,Acquisition Date,Mission,Frame,Image Type,Camera Type,Camera Resolution,Film Type,Generation,Segment Count,...,NW Corner Lat dec,NW Corner Long dec,NE Corner Lat dec,NE Corner Long dec,SE Corner Lat dec,SE Corner Long dec,SW Corner Lat dec,SW Corner Long dec,browse_url,geometry
0,DZB1214-500055L004001,1978-03-25,1214-5,4,1,KH-9 Lower Resolution Mapping,20 to 30 feet,9 inch by 18 inches,3,2,...,35.053,-111.899,34.905,-110.574,32.732,-110.939,32.874,-112.235,https://ims.cr.usgs.gov/browse/declassii/1214-...,"POLYGON ((-112.235 32.874, -110.939 32.732, -1..."
1,DZB1214-500055L005001,1978-03-25,1214-5,5,1,KH-9 Lower Resolution Mapping,20 to 30 feet,9 inch by 18 inches,3,2,...,34.403,-112.041,34.256,-110.725,32.082,-111.084,32.224,-112.372,https://ims.cr.usgs.gov/browse/declassii/1214-...,"POLYGON ((-112.372 32.224, -111.084 32.082, -1..."
2,DZB1214-500055L006001,1978-03-25,1214-5,6,1,KH-9 Lower Resolution Mapping,20 to 30 feet,9 inch by 18 inches,3,2,...,33.752,-112.182,33.607,-110.876,31.432,-111.23,31.572,-112.508,https://ims.cr.usgs.gov/browse/declassii/1214-...,"POLYGON ((-112.508 31.572, -111.23 31.432, -11..."
3,DZB1214-500055L007001,1978-03-25,1214-5,7,1,KH-9 Lower Resolution Mapping,20 to 30 feet,9 inch by 18 inches,3,1,...,33.102,-112.323,32.957,-111.026,30.782,-111.374,30.921,-112.645,https://ims.cr.usgs.gov/browse/declassii/1214-...,"POLYGON ((-112.645 30.921, -111.374 30.782, -1..."


## Step 1 : Download raw images

To download the raw images we will use [`usgsxplore`](https://github.com/adehecq/usgs_explorer) which is an python interface to the [USGS M2M API](https://m2m.cr.usgs.gov/) to search and download data available from the [Earth Explorer](https://earthexplorer.usgs.gov/) platform.

In [13]:
entity_ids = df["Entity ID"].to_list()

api = usgsxplore.API(username, token)
api.download("declassiii", entity_ids, output_dir=RAW_IMAGES)
api.logout()

history.dataquery.download_casa_grande_kh9mc_metadata(PROJECT_DIRECTORY)

if not os.path.exists(os.path.join(RAW_IMAGES, "quickviews")):
    hipp.tools.generate_quickviews(RAW_IMAGES, QUICKVIEW_FACTOR, max_workers=8)

## Step 2 : Preprocessing with ASPy

IMPORTANT : this part is not reproducable cause the pipeline is private. But if you have the code in your machine and you have setup the conda environment you can follow this guide to preprocess the data.
```bash
# activate your aspy conda environment
conda activate aspy

cd /home/godinlu/Documents/history-dataset-v2/casa_grande/kh9mc/1978_03_25

# create the preprocessing project
aspy create preproc

# create sym link to access the raw_images without moving them
for img in raw_images/*.tif; do ln -s "$PWD/$img" preproc/data/; done

# create sym link to acces the geojson file to avoid modifying the config
ln -s $PWD/images_footprint.geojson preproc/metadata.gpkg

# create an text file with all ids that will be preprocessed
echo -e "DZB1214-500055L004001\nDZB1214-500055L005001\nDZB1214-500055L006001\nDZB1214-500055L007001" > preproc/ids.txt

# go the preproc directory
cd preproc

# start the preprocessing
aspy preproc

```

In [10]:
!mkdir -p {PREPROCESSED_IMAGES}
!mv {PROJECT_DIRECTORY}/preproc/output/*/*_final.tif {PREPROCESSED_IMAGES}

mv: cannot stat '/home/godinlu/Documents/history-dataset-v2/casa_grande/kh9mc/1978_03_25/preproc/output/*/*_final.tif': No such file or directory


In [11]:
hipp.tools.generate_quickviews(PREPROCESSED_IMAGES, QUICKVIEW_FACTOR, max_workers=8)

Generating quickviews: 100%|██████████| 4/4 [00:01<00:00,  2.72image/s]
